In [150]:
import numpy as np
import pandas as pd 




N = 100
D = 5
K = 6*D



def Q(d,k):
    a = np.zeros((d+1,k+1))

    for _d in range(d+1):
        for _k in range(k+1):
            if (_k > 6*_d) or (_k < 2*_d):
                a[_d][_k] = 0.0
                continue
            if(_d == 1):
                a[_d][_k] = 1/5
                continue
            s = 0
            for i in range(2,7):
                if(_k>i):
                    s += a[_d-1][_k-i]

            a[_d][_k] = (1/5) * s

    return a[d][k]

def P(d,k):
    if k == 1:
        return 1.0 - (5.0/6.0)**d
    if (k > 6*d) or (k > 1 and k < 2*d):
        return 0.0
    return Q(d,k)*(5.0/6.0)**d


def generate_P_matrix(D):
	P_matrix = np.zeros((D+1,6*D+1))
	for i in range(1,D+1):
		for j in range(6*D+1):
			P_matrix[i][j] = P(i,j)
	return P_matrix




def EP(D=10):
    return 1+np.argmax(np.array([4*d*np.power(5/6,d) + (1-np.power(5/6,d)) for d in range(1,D+1) ]))




def G():
    def f(j,k):
        return (k > 0 and j>0)

    t = N + 6*D
    t = N
    a = np.zeros( ( t, t , 2 ))
    for i in range(1,t):
        for  j in range(1,t):
            l = [ probabilite[d][1]*(1 - a[j][i-1][0] ) + sum([ probabilite[d][k] * ( 1-a[j][i-k][0] * f(j,i-k) )
            for k in range (2*d,6*d + 1) ])  for d in range(1,D+1)]
            a[i][j][0] = max(l)
            a[i][j][1] = 1+np.argmax(l)

    return a



def esperance_gain():
    a = np.full((N+6*D,N+6*D),0)

    a[N: , :N] = 1
    a[:N , N: ] = -1

    optimal = np.zeros((N, N), dtype = int)

    for x in range(N - 1, -1, -1):
        for y in range(x, -1, -1):
            for i,j in {(x,y),(y,x)}:
                tmp = - probabilite[ :, 1: ].dot(a[j, (i + 1):(i + 6 * D + 1)])

                optimal[i, j] = tmp[1: ].argmax() + 1

                a[i, j] =  tmp[optimal[i, j]]

    return a,optimal



def matrice_gain(D=3):
    def f(i,j):
        if(i > j) : return 1
        if(j > i) : return -1
        if(i == j) : return 0
    
    
    return np.array([ [ f(j,i)  for  i  in range(6*D) ] for  j  in range(6*D) ] )

def eg_simul(D=3):
    def f(d1,d2):
        s_1 = s_2 = 0
        n = max(d1,d2)*6

        for i in range(1,6*d1+1):
            for j in range(1,6*d2+1):    

                if(j<i):
                    s_1 += probabilite[d1][i]

                if(i<j):
                    s_2 += probabilite[d2][j]
        p = (s_1 - s_2)/n
        return  np.round(p, 7)

    a = np.zeros((D+1,D+1))
    for d1 in range(1,D+1):
        for d2 in range(1,D+1):        
            a[d1][d2] = f(d1,d2)
    return a[1:,1:]




In [151]:
def EGsimuP(d1, d2 ):
    p_table_j1 = probabilite[d1]
    p_table_j2 = probabilite[d2]
    eg_j1=0
    for i in range(1, len(p_table_j1)):
        eg2=0
        for j in range(1, len(p_table_j2)):
            if i>j:
                eg2 += p_table_j2[j]
            if j>i:
                eg2 -= p_table_j2[j]

        eg_j1 += p_table_j1[i]*eg2

    return np.round(eg_j1 ,15)

def EGmatP(D):
    mat = np.zeros((D + 1, D + 1))

    for d1 in range(D + 1):
        for d2 in range(D + 1):
            mat[d1, d2] = EGsimuP(d1, d2)

    return mat[1:,1:]

In [152]:
probabilite = generate_P_matrix(D)
p = pd.DataFrame(data=probabilite[1:,1:],index=["d="+str(i) for i in np.arange(1,D+1)],columns= ["k="+str(i) for i in np.arange(1,6*D+1)] )
p.to_csv('./probabiltes.csv')

In [153]:
m = EGmatP(D).T

In [155]:
#Notre Probleme

nb_variables = D
prob = LpProblem("Dices D="+str(D),LpMaximize)
var_s = ['q'+str(i) for i in range(1,D+1) ]
variables = LpVariable.dicts("D",var_s,lowBound=0,cat='Continuous')

coefs_obj =  m.T

l = []
for i in range(len(coefs_obj)):
    for j in range(len(coefs_obj[0])):
        l.append(coefs_obj[i][j]*variables[var_s[j]])



prob += lpSum(l)


prob +=  lpSum(np.array(list(variables.values())))  == 1


print(prob)

"""
prob.solve(pulp.GLPK_CMD())

print(LpStatus[prob.status])

for v in prob.variables():
    print(v.name,":", v.varValue)

print("z=",value(prob.objective))

"""
print()

Dices D=5:
MAXIMIZE
0.556091392318245*D_q1 + -0.07342035322359301*D_q2 + -0.280147367017223*D_q3 + -0.22270080383452798*D_q4 + 0.02017713175709899*D_q5 + 0.0
SUBJECT TO
_C1: D_q1 + D_q2 + D_q3 + D_q4 + D_q5 = 1

VARIABLES
D_q1 Continuous
D_q2 Continuous
D_q3 Continuous
D_q4 Continuous
D_q5 Continuous




In [113]:
nb_variables = 4 
prob = LpProblem("Dices",LpMaximize)
var_s = ['p1','p2','p3','p4']
variables = LpVariable.dicts("",var_s,lowBound=0,cat='Continuous')

coefs_obj = -1*np.array([[0,-2,+1,+1],[+2,0,-2,+1],[-1,+2,0,-2],[-1,-1,+2,0]])

l = []
for i in range(len(coefs_obj)):
    for j in range(len(coefs_obj[0])):
        l.append(coefs_obj[i][j]*variables[var_s[j]])

        
        

prob += lpSum(l)


prob +=  lpSum(np.array(list(variables.values())))  <= 1
prob += -lpSum(np.array(list(variables.values())))  <= -1

prob.solve()
print(LpStatus[prob.status])

for v in prob.variables():
    print(v.name,":", v.varValue)

print("z=",value(prob.objective))

print()
print(prob)

Optimal
_p1 : 0.0
_p2 : 1.0
_p3 : 0.0
_p4 : 0.0
z= 1.0

Dices:
MAXIMIZE
0*_p1 + 1*_p2 + -1*_p3 + 0*_p4 + 0
SUBJECT TO
_C1: _p1 + _p2 + _p3 + _p4 <= 1

_C2: - _p1 - _p2 - _p3 - _p4 <= -1

VARIABLES
_p1 Continuous
_p2 Continuous
_p3 Continuous
_p4 Continuous

